<a href="https://colab.research.google.com/github/Riiishaab/Sign_Language_Detector/blob/main/Sign_Lang_Model_Deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install required packages

In [2]:
!pip install -q fastai opencv-python

# Load Full Model

In [9]:
import cv2
from fastai.vision.all import *
from google.colab import files
import numpy as np
from google.colab.patches import cv2_imshow  # For displaying images in Colab

# Show the upload button and prompt user to upload their model file
print("Please upload your model file (e.g., sign_lang_model.pkl):")
uploaded = files.upload()  # This will show the upload button

# Get the filename of the uploaded file
model_filename = list(uploaded.keys())[0]

# Load your exported model
learn = load_learner(model_filename)
print("Model loaded successfully!")


Please upload your model file (e.g., sign_lang_model.pkl):


Saving sign_lang_model.pkl to sign_lang_model (1).pkl
Model loaded successfully!


# Real Time Web-Cam Deployment


#### -> Allow Camera permissions for Colab.
####-> Wait for 5 seconds for web cam to capture pics.
####-> It does real time Sign Prediction.
####-> Run the cell as many times as you want

In [ ]:
import cv2
from fastai.vision.all import *
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from PIL import Image
import io
from google.colab.patches import cv2_imshow
import time

# HAGRID label mapping (label number to class name)
hagrid_classes = [
    "call", "dislike", "fist", "four", "like", "mute", "no_gesture", "ok", "one", "palm",
    "peace", "peace_inverted", "rock", "stop", "stop_inverted", "three", "three2", "two_up", "two_up_inverted"
]

def take_photo_and_predict(learn):
    # JavaScript to access webcam and take photo
    js = Javascript('''
    async function takePhoto() {
      const div = document.createElement('div');
      const video = document.createElement('video');
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize output to video dimensions
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for 5 seconds to let the camera initialize
      await new Promise(resolve => setTimeout(resolve, 5000));

      // Create a canvas to capture a frame
      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);

      stream.getVideoTracks()[0].stop();
      div.remove();

      return canvas.toDataURL('image/jpeg');
    }
    ''')

    print("Accessing webcam... Please allow camera access when prompted.")
    display(js)
    print("Waiting 5 seconds for webcam to initialize...")
    data = eval_js('takePhoto()')

    # Process the image data
    binary = b64decode(data.split(',')[1])
    image = Image.open(io.BytesIO(binary))

    # Display the original image
    print("Image captured from webcam:")
    display(image)

    # Make prediction
    pred_class, pred_idx, probs = learn.predict(image)

    # Map label number to class name using hagrid_classes
    pred_label_num = int(pred_idx)
    pred_label_name = hagrid_classes[pred_label_num]

    # Display result with prediction overlay
    img_np = np.array(image)
    frame = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)
    cv2.putText(frame, f"{pred_label_name}: {probs[pred_label_num]:.4f}",
                (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    print("\nImage with prediction:")
    cv2_imshow(frame)
    print(f"Prediction: {pred_label_name} (label {pred_label_num}) with confidence {probs[pred_label_num]:.4f}")

    return image, pred_label_name, probs[pred_label_num]

# Use the function with your loaded model
take_photo_and_predict(learn)
